In [ ]:
import threading
import Queue
import time
from pyspark.streaming import StreamingContext

In [ ]:
BATCH_INTERVAL = 10   # How frequently to update (seconds)
WINDOWS_LENGTH = 60   # the duration of the window
SLIDING_INTERVAL = 20 # the interval at which the window operation is performed

In [ ]:
#To setup the streaming data
ssc = StreamingContext(sc, BATCH_INTERVAL)
#ssc.checkpoint("checkpoint")

# Create a DStream that will connect to hostname:port, like localhost:9999
dstream = ssc.socketTextStream("localhost", 8889)
dstreamwin=dstream.window(WINDOWS_LENGTH, SLIDING_INTERVAL)

dstream_tweets=dstream.map(lambda post: get_json(post))
    
dstream_tweets.pprint()

In [ ]:
ssc.start()
ssc.awaitTermination()

In [ ]:
dstream_tweets=dstream.map(lambda post: get_json(post))\
     .filter(lambda post: post != False)\
     .filter(lambda post: 'created_at' in post)\
     .map(lambda post: (get_coord2(post)[0],get_coord2(post)[1],post["text"]))\
     .filter(lambda tpl: tpl[0] != 0)\
     .filter(lambda tpl: tpl[2] != '')\
     .map(lambda tpl: (tpl[0],tpl[1],tokenize(tpl[2])))\
     .map(lambda tpl:(tpl[0],tpl[1],tpl[2],doc2vec(tpl[2])))